# Choropleth Map for Annual Change

In [ ]:
import plotly.express as px
import pandas as pd

### Load Data

In [ ]:
df = pd.read_csv("../output/state_growth_rates.csv")
df.head(10)

### Choropleth Animation through out the Year

In [ ]:
import plotly.graph_objects as go

# Scrape state centroids from Google's public dataset
tables = pd.read_html('https://developers.google.com/public-data/docs/canonical/states_csv')
state_coords = tables[0]
state_coords.columns = ['state', 'latitude', 'longitude', 'name']

# Merge centroids into house price data
df = df.merge(state_coords[['state', 'latitude', 'longitude']], left_on='Abbreviation', right_on='state')

# Create animated choropleth with year slider
fig = px.choropleth(
    df,
    locations='Abbreviation',
    locationmode='USA-states',
    color='Annual Change (%)',
    color_continuous_scale='RdYlGn',
    range_color=(-20, 20),
    scope='usa',
    animation_frame='Year',
    labels={'Annual Change (%)': 'Annual Change (%)'},
    title='Annual House Price Change by State'
)

# State label trace (uses first year's data for coordinates)
coords = df.drop_duplicates(subset='Abbreviation')
label_trace = go.Scattergeo(
    locationmode='USA-states',
    lon=coords['longitude'],
    lat=coords['latitude'],
    text=coords['Abbreviation'],
    mode='text',
    textfont=dict(size=8, color='white'),
    showlegend=False,
    hoverinfo='skip'
)

# Add labels to the base figure
fig.add_trace(label_trace)

# Add labels to every animation frame so they persist
for frame in fig.frames:
    frame.data = (*frame.data, label_trace)

fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0})
fig.show()